<a href="https://colab.research.google.com/github/Kibika/speech-to-text/blob/speech_to_text/notebooks/resize_standardize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import wave
import os
import librosa

In [ ]:
Path_to_train = "train\wav"
subfolders = os.listdir(Path_to_train)
data = []
for s in subfolders:
    files = os.listdir(Path_to_train + "/" +s)
    data.extend([Path_to_train + "/" + s+ "/" + f for f in files])
data[:10]

In [ ]:
#read text from every transcription audio
def read_text( text_path):
    text = []
    with open(text_path) as fp:
        line = fp.readline()
        while line:
        # TODO: fix spaces in in amharic text
            text.append(line)
            line = fp.readline()
    return text

In [ ]:
#extract the transcription and the label 
label=[]
transcriptions = []
for t in text:
    sp = t.split("\t")
    sp = sp.strip("\n")
    if len(sp) > 1:
        label.append(sp[0])
        transcriptions.append(sp[1])
transcriptions[:10]

In [ ]:
#get audio path , every path must corespond to transcription , get the transprion in the doc and append to audio path 
audio_path=[0]*len(transcriptions)
for d in data:
    _d = d.strip(".wav")
    sp = _d.split("/")[2]
    index = label.index(sp)
    audio_path[index] = d

In [ ]:
#calculate duration 
duration_of_recordings=[]
for d in audio_path:
    audio, fs = librosa.load(d, sr=None)
    duration_of_recordings.append(float(len(audio)/fs))

In [ ]:
import pandas as pd 
data=pd.DataFrame({'key': audio_path,'text': transcriptions, 'duration':duration_of_recordings})

# Resize and standardize
We resize all the audio samples to have the same length by either extending its duration by padding it with silence, or by truncating it.
We use the right padding method

Standardize the sample rate.


In [ ]:
# used to load audio file
#specifying sample rate will resize all the files i.e Audio will be automatically resampled to the given rate
class Loader:
  def __init__(self, sample_rate,duration,mono):
    self.sample_rate=sample_rate
    self.duration=duration
    self.mono=mono

  def load_signal(self, filepath):
    signal=librosa.load(filepath,
                        sr=self.sample_rate,
                        duration=self.duration,
                        mono=self.mono)[0]    #librosa returns 2D array (signal,sample_rate) pick the signal here
    return signal

  def load_sample_rate(self,filepath):
    sample_rate=librosa.load(filepath,
                        sr=self.sample_rate,
                        duration=self.duration,
                        mono=self.mono)[1]    #librosa returns 2D array (signal,sample_rate) pick the sample rate here
    return sample_rate

  def load(self,filepath):     #get both signal and sample rate in one
    aud=librosa.load(filepath,
                        sr=self.sample_rate,
                        duration=self.duration,
                        mono=self.mono)
    return aud

In [ ]:
#resizes audios to have same length
class Padder:
  def __init__(self,mode="constant"):
    self.mode=mode
  def left_pad(self,array,num_missing_items):
    padded_array=np.pad(array,
                        (num_missing_items, 0),
                        mode=self.mode)
    return padded_array
  def right_pad(self,array,num_missing_items):
    padded_array=np.pad(array,
                        (0,num_missing_items),
                        mode=self.mode)
    return padded_array
  

In [ ]:
class PreprocessingPipeline:
  '''Processes audio files in a directory by applying the following steps
    1. Load the data, convert to mono and resample sampling rate
    2. Pad the audio
  '''
  def __init__(self):
    self.padder=None
    # self.saver=None
    self._loader=None
    # self._num_expected_samples=None


 
  # def loader(self):
  #   return self._loader


 
  # def loader(self,loader):
  #   self.loader=loader
  #   self._num_expected_samples=int(loader.sample_rate*loader.duration)



  def process(self,audio_files_directory):
    for root, directories, files in os.walk(audio_files_directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            self._process_file(filepath)
            print(f"Processed file {filepath}")
    
  def _process_file(self,filepath):
    signal=self.loader.load_signal(filepath)
    if self._is_padding_necessary(signal):
      signal=self._apply_padding(signal)

  def _is_padding_necessary(self,signal):
    num_expected_samples=int(loader.sample_rate*loader.duration)
    if len(signal) < num_expected_samples:
      return True
    return False

  def apply_padding(self,signal):
    num_missing_samples=self._num_expected_samples - len(signal)
    padded_signal = self.padder.right_pad(signal, num_missing_samples)
    return padded_signal


In [ ]:
DURATION=0.74
SAMPLE_RATE=22050
MONO=True

In [ ]:
loader=Loader(SAMPLE_RATE, DURATION, MONO)
padder=Padder()

In [ ]:
preprocessing_pipeline=PreprocessingPipeline()
preprocessing_pipeline.loader=loader
preprocessing_pipeline.padder=padder

In [ ]:
preprocessing_pipeline.process(Path_to_train)